In [1]:
with open('input.txt', 'r', encoding='utf-8') as file:
  text = file.read()

In [2]:
print(len(text))

1115394


In [3]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [4]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [5]:
# Create mapping for characters

itos = {i:s for i, s in enumerate(chars)}
stoi = {s:i for i, s in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s] # take a string output a list of integer
decode = lambda l: ''.join([itos[i] for i in l]) # take a list of integer output a string

print(encode('hello world'))
print(decode(encode('hello world')))

[46, 43, 50, 50, 53, 1, 61, 53, 56, 50, 42]
hello world


In [6]:
# Encode the entire text dataset and store it in torch.Tensor

import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000]) # what data will look like to gpt

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [7]:
# Splitting data into train and validation sets

n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [8]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [9]:
x = train_data[:block_size]
y = train_data[1:block_size + 1]

x, y

(tensor([18, 47, 56, 57, 58,  1, 15, 47]),
 tensor([47, 56, 57, 58,  1, 15, 47, 58]))

In [10]:
for t in range(block_size):
  context = x[:t+1]
  target = y[t]
  print(f'When input is {context} the target is {target}')

When input is tensor([18]) the target is 47
When input is tensor([18, 47]) the target is 56
When input is tensor([18, 47, 56]) the target is 57
When input is tensor([18, 47, 56, 57]) the target is 58
When input is tensor([18, 47, 56, 57, 58]) the target is 1
When input is tensor([18, 47, 56, 57, 58,  1]) the target is 15
When input is tensor([18, 47, 56, 57, 58,  1, 15]) the target is 47
When input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target is 58


In [11]:
torch.manual_seed(1337)
batch_size = 4  # how many independent sequences will be processed in parallel?
block_size = 8  # what is the maximum context length for prediction?

def get_batch(split):
  # generate small batch of data of inputs x and target y
  data = train_data if split == 'train' else val_data
  ix = torch.randint(len(train_data) - block_size, (batch_size, ))
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1: i+block_size+1] for i in ix])
  return x, y

xb, yb = get_batch('train')
print('Inputs:')
print(xb.shape)
print(xb)
print('Targets:')
print(yb.shape)
print(yb)

print('-'*10)

for b in range(batch_size):       # batch dimension
  for t in range(block_size):     # time dimension
    context = xb[b, :t+1]
    target = yb[b, t]
    print(f' When input is {context.tolist()}, target is: {target}')


Inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
Targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----------
 When input is [24], target is: 43
 When input is [24, 43], target is: 58
 When input is [24, 43, 58], target is: 5
 When input is [24, 43, 58, 5], target is: 57
 When input is [24, 43, 58, 5, 57], target is: 1
 When input is [24, 43, 58, 5, 57, 1], target is: 46
 When input is [24, 43, 58, 5, 57, 1, 46], target is: 43
 When input is [24, 43, 58, 5, 57, 1, 46, 43], target is: 39
 When input is [44], target is: 53
 When input is [44, 53], target is: 56
 When input is [44, 53, 56], target is: 1
 When input is [44, 53, 56, 1], target is: 58
 When input is [44, 53, 56, 1, 58], target is: 46
 W

In [19]:
import torch.nn as nn 
from torch.nn import functional as F 
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size) -> None:
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B, T) tensors of integers
        logits = self.token_embedding_table(idx) # (B, T, C)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus on last time step
            logits = logits[:, -1, :]   # becomes (B, T)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=1)
            # sample from probs
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [26]:
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

torch.Size([256, 65])
tensor(4.7633, grad_fn=<NllLossBackward0>)


In [27]:
idx = torch.zeros((1, 1), dtype=torch.long)
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist()))


;b
cFbI;cFlv',PF3CIUZXZGdYWoH!lkDiLNsO;';mjS?U-EmpI;LHuqldqSyq?:',;i:aW&zPcw
JIPHWRZ.dC&LmsGU.UI:FMM


In [28]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [29]:
batch_size = 32
for steps in range(10000):
    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.4859490394592285


In [31]:
print(decode(m.generate(idx, max_new_tokens=400)[0].tolist()))


ndir'US fakecheng wads tt h.
SCI if G m gliofatighyonthas: onouspe attotathiso fre.
M:
CKI?
Bur d toortXn stearellavejou'CINBENBUCER gon dsoomerimoulllot nderinds facarkiing br cth ol we
Hend b
BEL:
pen ancestwixt congrnonense fof wouz'

NELOr that gh liust p,quremet m
IORun seagefathe t ty, cin foind s d an wepe ca'BUS:

Wt s s sh'd are.
BONNTind t--the po und I IUThep.
Wh athrle wo mbef mu's nu.


In [33]:
# Self-attention

torch.manual_seed(1337)
B, T, C = 4, 8, 32
x = torch.randn(B, T, C)

head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)      # (B, T, 16)
q = query(x)    # (B, T, 16)
wei = q @ k.transpose(-2, -1)   # (B, T, 16) @ (B, 16, T) --> (B, T, T)

tril = torch.tril(torch.ones(T, T))
#wei = wei.masked_fill(tril == 0, float('-inf')) 
wei = F.softmax(wei, dim=1)

v = value(x)
out = wei @ v

out.shape

torch.Size([4, 8, 32])